In [1]:
# Meeting Booking Text Examples
# Collection of common phrases used for booking meetings

meeting_booking_examples = [
    # Immediate/Today
    "I want to book a meeting at 5 PM today",
    "Can we schedule a meeting for 3 PM this afternoon?",
    "I need to set up a meeting at 2:30 PM today",
    
    # Specific Time Requests
    "I want to book a meeting at 5 PM",
    "Let's schedule a meeting for 10 AM tomorrow",
    "I'd like to arrange a meeting at 1 PM on Friday",
    "Can we meet at 9:30 AM next Tuesday?",
    
    # Next Week Requests
    "I want next week 4 day meeting",
    "I need to schedule a meeting for next week Wednesday",
    "Let's book a meeting for next Monday at 11 AM",
    "I want to arrange a meeting next week Friday",
    
    # Duration-Specific
    "I need a 30-minute meeting next Thursday",
    "Can we schedule a 1-hour meeting for next week?",
    "I want to book a quick 15-minute meeting tomorrow",
    
    # Multiple Participants
    "I want to schedule a team meeting for next week",
    "Let's book a client meeting for Friday at 3 PM",
    "I need to arrange a project meeting next Tuesday",
    
    # Flexible Timing
    "I want to book a meeting sometime next week",
    "Can we schedule a meeting for early next week?",
    "I need to set up a meeting before Friday"
]


In [40]:
# Meeting Booking Text Examples
# Collection of common phrases used for booking meetings

meeting_booking_examples = [
    # Immediate/Today
    "I want to book a meeting at 5 PM today",
    "Can we schedule a meeting for 3 PM this afternoon?",
    "I need to set up a meeting at 2:30 PM today",
    
    # Specific Time Requests
    "I want to book a meeting at 5 PM",
    "Let's schedule a meeting for 10 AM tomorrow",
    "I'd like to arrange a meeting at 1 PM on Friday",
    "Can we meet at 9:30 AM next Tuesday?",
    
    # Next Week Requests
    "I want next week 4 day meeting",
    "I need to schedule a meeting for next week Wednesday",
    "Let's book a meeting for next Monday at 11 AM",
    "I want to arrange a meeting next week Friday",
    
    # Duration-Specific
    "I need a 30-minute meeting next Thursday",
    "Can we schedule a 1-hour meeting for next week?",
    "I want to book a quick 15-minute meeting tomorrow",
    
    # Multiple Participants
    "I want to schedule a team meeting for next week",
    "Let's book a client meeting for Friday at 3 PM",
    "I need to arrange a project meeting next Tuesday",
    
    # Flexible Timing
    "I want to book a meeting sometime next week",
    "Can we schedule a meeting for early next week?",
    "I need to set up a meeting before Friday"
]
  
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta, MO, FR, TU, WE, TH

WEEKDAYS = {
    'monday': MO,
    'tuesday': TU,
    'wednesday': WE,
    'thursday': TH,
    'friday': FR,
}

def calculate_meeting_time(params, reference_date=None):
    """
    Calculate meeting start and end datetime from parameters.
    params: dict with keys 'date', 'time', 'duration' (all optional)
    reference_date: datetime, defaults to today
    Returns: dict with 'start', 'end' (datetime objects)
    """
    if reference_date is None:
        reference_date = datetime.now()
    # Parse date
    date_str = params.get('date', '').lower()
    time_str = params.get('time', '')
    duration_str = params.get('duration', '')

    # Default start date
    start_date = reference_date
    # Handle relative dates
    if 'next week' in date_str:
        start_date += relativedelta(weeks=+1, weekday=MO(+1))
    elif 'tomorrow' in date_str:
        start_date += timedelta(days=1)
    elif 'today' in date_str:
        pass
    else:
        for wd in WEEKDAYS:
            if wd in date_str:
                # Find next weekday
                start_date += relativedelta(weekday=WEEKDAYS[wd](+1))
                break
    # Parse time
    match = re.search(r'(\d{1,2})(:(\d{2}))?\s*(am|pm)?', time_str.lower())
    hour, minute = 9, 0  # default
    if match:
        hour = int(match.group(1))
        minute = int(match.group(3) or 0)
        if match.group(4) == 'pm' and hour < 12:
            hour += 12
        elif match.group(4) == 'am' and hour == 12:
            hour = 0
    start_dt = start_date.replace(hour=hour, minute=minute, second=0, microsecond=0)
    # Parse duration
    duration_minutes = 30  # default
    dur_match = re.search(r'(\d+)[-\s]?(minute|hour)', duration_str.lower())
    if dur_match:
        val = int(dur_match.group(1))
        if dur_match.group(2) == 'hour':
            duration_minutes = val * 60
        else:
            duration_minutes = val
    end_dt = start_dt + timedelta(minutes=duration_minutes)
    return {'start': start_dt, 'end': end_dt}

from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
        model="Qwen/Qwen3-4B",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        api_key="abc-123",  # if you prefer to pass api key in directly instaed of using env vars
        base_url="http://129.212.190.146:8000/v1/",

        # organization="...",
        # other params...
        
    )
from pydantic import BaseModel, Field

class MeetingParams(BaseModel):
    """
    Parameters for booking a meeting, optimized for LLM extraction from natural language queries.

    - 'temporal_expressions': Accepts natural language date references in English (e.g., 'next Monday', 'tomorrow').
    - 'time': Accepts natural language time references in English (e.g., '3 PM', '10 AM', 'afternoon').
    - 'duration': Accepts durations in English (e.g., '30 minutes', '1 hour', 'quick').
    - 'type': Accepts meeting types such as 'team', 'client', 'project', 'HR'.

    Notes:
    - All values must be expressed in English only.
    - The workweek is considered Monday to Friday.
    - Designed to support LLM extraction logic for scheduling assistants.
    """

    temporal_expressions: str = Field(
        default="",
        description="Natural language date (English only), e.g., 'next Monday', 'tomorrow'"
    )
    time: str = Field(
        default="",
        description="Natural language time (English only), e.g., '3 PM', '10 AM', 'afternoon'"
    )
    duration: str = Field(
        default="",
        description="Meeting duration (English only), e.g., '30 minutes', '1 hour', 'quick'"
    )
    type: str = Field(
        default="",
        description="Type of meeting, e.g., 'team', 'client', 'project', 'HR'"
    )

new_llm=llm.with_structured_output(MeetingParams)
example="book half hr meeting next week "
result = new_llm.invoke(example)
meeting_times = calculate_meeting_time(result.model_dump())

In [41]:
result

MeetingParams(temporal_expressions='next week', time='next week', duration='half hr', type='meeting')

In [42]:
meeting_times

{'start': datetime.datetime(2025, 7, 20, 9, 0),
 'end': datetime.datetime(2025, 7, 20, 9, 30)}

In [14]:
test_examples = [
    # "Please book a video call with the client for next Wednesday at 3 PM.",
    # "Can we set up a quick 15-minute meeting tomorrow to catch up on the project?",
    # "Please arrange a team meeting next Monday to review the quarterly goals.",
    # "Schedule an emergency call ASAP to discuss the system outage.",
    "book half hr meeting next week ",
]

for example in test_examples:
    
    print(f"Input: {example}\nParams: {result}\nMeeting Times: {meeting_times}\n")

Input: book half hr meeting next week 
Params: date='2023-10-25' time='14:00' duration='30' type='meeting'
Meeting Times: {'start': datetime.datetime(2025, 7, 20, 14, 0), 'end': datetime.datetime(2025, 7, 20, 14, 30)}



In [64]:
import re
from datetime import datetime, timedelta
from dateutil import parser as dateutil_parser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
import json

# --- 1. DEFINE THE PARAMETER SCHEMA ---
# This is the "form" the LLM needs to fill out to use our tool.
# The docstrings here are critical, as they are the instructions for the LLM.
class MeetingParams(BaseModel):
    """The parameters required to book a meeting."""
    temporal_expression: str = Field(
        ..., # '...' makes this field required
        description="All temporal information for the meeting, like 'next Wednesday at 4pm' or 'tomorrow afternoon'."
    )
    duration: str = Field(
        default="30 minutes",
        description="The meeting's duration, e.g., '1 hour', 'quick', or 'half an hour'."
    )
    
# --- 2. CREATE THE STANDARDIZED TOOL FUNCTION ---
# This is the core, standardized function. It is self-contained, testable,
# and handles all the business logic. It knows NOTHING about the LLM.
def book_meeting(params: MeetingParams, reference_date: datetime = None) -> dict:
    """
    Validates meeting parameters and calculates the final start and end datetimes.

    This function acts as the final, reliable step to schedule a meeting after
    an AI has extracted the necessary parameters from a user's request.

    Args:
        params (MeetingParams): A Pydantic model containing the extracted temporal,
                                duration, and type information.
        reference_date (datetime): The datetime to consider as 'now'. Used for
                                   consistent testing. Defaults to datetime.now().

    Returns:
        dict: A dictionary containing the meeting_type, and the calculated
              start and end datetime objects.
    """
    if reference_date is None:
        reference_date = datetime.now()

    # --- Duration Logic ---
    duration_str = params.duration.lower()
    duration_minutes = 30  # Default
    friendly_durations = {"half hr": 30, "half an hour": 30, "quick": 15, "an hour": 60}
    for phrase, mins in friendly_durations.items():
        if phrase in duration_str:
            duration_minutes = mins
            break
    else:
        dur_match = re.search(r'(\d+)\s*(minute|hour)', duration_str)
        if dur_match:
            val, unit = int(dur_match.group(1)), dur_match.group(2)
            duration_minutes = val * 60 if unit == 'hour' else val

    # --- Date & Time Logic ---
    time_expr = params.temporal_expression.lower()
    friendly_times = {"afternoon": "3 PM", "morning": "10 AM", "evening": "6 PM", "noon": "12 PM"}
    for keyword, replacement in friendly_times.items():
        if keyword in time_expr and not any(char.isdigit() for char in time_expr):
            time_expr = time_expr.replace(keyword, replacement)

    # Use a sensible default (9 AM of the reference_date)
    default_start_time = reference_date.replace(hour=9, minute=0, second=0, microsecond=0)
    
    try:
        # dateutil.parser is the standard for robustly parsing date strings
        start_dt = dateutil_parser.parse(time_expr, default=default_start_time)
        # If the user only said "next Friday" (no time), default to 9 AM, not the current time.
        if not re.search(r'\d{1,2}\s*([ap]m|:)', time_expr):
             start_dt = start_dt.replace(hour=9, minute=0)
    except dateutil_parser.ParserError:
        # Graceful fallback if date is un-parseable
        start_dt = default_start_time
        if start_dt < reference_date: # If 9am today is already past
             start_dt += timedelta(days=1)
    
    end_dt = start_dt + timedelta(minutes=duration_minutes)

    return {
        #"meeting_type": params.meeting_type,
        "start_time": start_dt,
        "end_time": end_dt,
        "status": "SUCCESS"
    }


# --- 3. CREATE THE ORCHESTRATOR/AGENT ---
# This class connects the user query to the LLM and the LLM to the tool.
class MeetingBookingAgent:
    def __init__(self, model):
        # STANDARD: The LLM is a dependency passed into the agent.
        self.llm = model.with_structured_output(MeetingParams)

    def process_request(self, user_query: str, reference_date: datetime = None) -> str:
        """
        Orchestrates the entire process from query to final result.
        """
        print(f"➡️ Processing Query: '{user_query}'")
        try:
            # Step 1: LLM extracts parameters from the query
            extracted_params = self.llm.invoke(user_query)
            print(f"✅ LLM Extraction Successf
            
                  ul: {extracted_params.model_dump()}")

            # Step 2: Execute the standard tool function with the extracted params
            result = book_meeting(params=extracted_params, reference_date=reference_date)
            print(f"🛠️ Tool `book_meeting` Executed Successfully.")

            # Step 3: Format a user-friendly response
            confirmation = (
                f"🎉 Meeting Confirmed!\n"
              
                f"   - When: {result['start_time'].strftime('%A, %B %d at %I:%M %p')}"
            )
            return confirmation

        except Exception as e:
            print(f"❌ An error occurred: {e}")
            return "I'm sorry, I couldn't understand that request. Could you please rephrase it?"

# --- USAGE EXAMPLE ---
if __name__ == "__main__":
    # Setup LLM and Agent
    llm = ChatOpenAI(model="Qwen/Qwen3-4B", temperature=0, api_key="abc-123", base_url="http://129.212.190.146:8000/v1/")
    agent = MeetingBookingAgent(model=llm)

    # Use a fixed date for consistent testing
    test_date = datetime(2025, 7, 2, 10, 0, 0) # Pretend it's Monday, 10 AM

    # --- Test Cases ---
    print("\n--- Test Case 1: Complex Request ---")
    response_1 = agent.process_request("Can we book a client meeting for next Wednesday afternoon? Make it half an hour.", reference_date=test_date)
    print(response_1)

    print("\n--- Test Case 2: Simple Request ---")
    response_2 = agent.process_request("meeting tomorrow 2pm", reference_date=test_date)
    print(response_2)

    print("\n--- Test Case 3: Vague Request ---")
    response_3 = agent.process_request("I need a quick project meeting sometime next week", reference_date=test_date)
    print(response_3)



--- Test Case 1: Complex Request ---
➡️ Processing Query: 'Can we book a client meeting for next Wednesday afternoon? Make it half an hour.'
✅ LLM Extraction Successful: {'temporal_expression': 'next Wednesday afternoon', 'duration': 'half an hour'}
🛠️ Tool `book_meeting` Executed Successfully.
🎉 Meeting Confirmed!
   - When: Monday, July 21 at 09:00 AM

--- Test Case 2: Simple Request ---
➡️ Processing Query: 'meeting tomorrow 2pm'
✅ LLM Extraction Successful: {'temporal_expression': 'tomorrow 2pm', 'duration': '2 hours'}
🛠️ Tool `book_meeting` Executed Successfully.
🎉 Meeting Confirmed!
   - When: Monday, July 21 at 09:00 AM

--- Test Case 3: Vague Request ---
➡️ Processing Query: 'I need a quick project meeting sometime next week'
✅ LLM Extraction Successful: {'temporal_expression': 'next week', 'duration': 'quick'}
🛠️ Tool `book_meeting` Executed Successfully.
🎉 Meeting Confirmed!
   - When: Monday, July 21 at 09:00 AM


In [65]:
book_meeting

<function __main__.book_meeting(params: __main__.MeetingParams, reference_date: datetime.datetime = None) -> dict>

In [66]:
meeting_booking_examples

['I want to book a meeting at 5 PM today',
 'Can we schedule a meeting for 3 PM this afternoon?',
 'I need to set up a meeting at 2:30 PM today',
 'I want to book a meeting at 5 PM',
 "Let's schedule a meeting for 10 AM tomorrow",
 "I'd like to arrange a meeting at 1 PM on Friday",
 'Can we meet at 9:30 AM next Tuesday?',
 'I want next week 4 day meeting',
 'I need to schedule a meeting for next week Wednesday',
 "Let's book a meeting for next Monday at 11 AM",
 'I want to arrange a meeting next week Friday',
 'I need a 30-minute meeting next Thursday',
 'Can we schedule a 1-hour meeting for next week?',
 'I want to book a quick 15-minute meeting tomorrow',
 'book half hr meeting next week',
 'I want to schedule a team meeting for next week',
 "Let's book a client meeting for Friday at 3 PM",
 'I need to arrange a project meeting next Tuesday',
 'I want to book a meeting sometime next week',
 'Can we schedule a meeting for early next week?',
 'I need to set up a meeting before Friday']

In [67]:
response_3 = agent.process_request(meeting_booking_examples[0],reference_date=test_date)
print(response_3)

➡️ Processing Query: 'I want to book a meeting at 5 PM today'
✅ LLM Extraction Successful: {'temporal_expression': '5 PM today', 'duration': '1 hour'}
🛠️ Tool `book_meeting` Executed Successfully.
🎉 Meeting Confirmed!
   - When: Monday, July 21 at 09:00 AM


In [ ]:

meeting_booking_examples = [
    # Immediate/Today
    "I want to book a meeting at 5 PM today",
    "Can we schedule a meeting for 3 PM this afternoon?",
    "I need to set up a meeting at 2:30 PM today",
    
    # Specific Time Requests
    "I want to book a meeting at 5 PM",
    "Let's schedule a meeting for 10 AM tomorrow",
    "I'd like to arrange a meeting at 1 PM on Friday",
    "Can we meet at 9:30 AM next Tuesday?",
    
    # Next Week Requests
    "I want next week 4 day meeting",
    "I need to schedule a meeting for next week Wednesday",
    "Let's book a meeting for next Monday at 11 AM",
    "I want to arrange a meeting next week Friday",
    
    # Duration-Specific
    "I need a 30-minute meeting next Thursday",
    "Can we schedule a 1-hour meeting for next week?",
    "I want to book a quick 15-minute meeting tomorrow",
    
    # Multiple Participants
    "I want to schedule a team meeting for next week",
    "Let's book a client meeting for Friday at 3 PM",
    "I need to arrange a project meeting next Tuesday",
    
    # Flexible Timing
    "I want to book a meeting sometime next week",
    "Can we schedule a meeting for early next week?",
    "I need to set up a meeting before Friday"
]
  